# Configuracao spark

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("datamaster_case_gustavo")
    # Usar todos os cores locais disponíveis;
    .master("local[*]")
    # memória do driver (~70% da RAM total)
    .config("spark.driver.memory", "20g")
    # tamanho de partição padrão (OBS: ajustar depois baseado no tamanho final)
    .config("spark.sql.shuffle.partitions", "200")
    # compressão em disco
    .config("spark.sql.parquet.compression.codec", "snappy")
    # para evitar broadcast grande nos joins
    .config("spark.sql.autoBroadcastJoinThreshold", "50MB")
    .getOrCreate()
)

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [3]:
base_path = r"C:\Users\Gustavo\Downloads\datamaster\dados\parquet"  # ajuste para o seu caminho

df_logs = spark.read.parquet(f"{base_path}/user_logs.parquet")
df_transactions   = spark.read.parquet(f"{base_path}/transactions.parquet")
df_members  = spark.read.parquet(f"{base_path}/members.parquet")

# Analisando bases

## logs

### dictionary

Definition: daily user logs describing listening behaviors of a user. Data collected until 3/31/2017.

* msno: user id
* date: format %Y%m%d
* num_25: # of songs played less than 25% of the song 
length
* num_50: # of songs played between 25% to 50% of the 
song length
* num_75: # of songs played between 50% to 75% of of
the song length
* num_985: # of songs played between 75% to 98.5% of 
the song length
* num_100: # of songs played over 98.5% of the song 
length
* num_unq: # of unique songs played
* total_secs: total seconds played





### general info

In [3]:
df_logs.count()

26758971

In [ ]:
df_logs.printSchema()

root
 |-- msno: string (nullable = true)
 |-- safra: integer (nullable = true)
 |-- num_25: double (nullable = true)
 |-- num_50: double (nullable = true)
 |-- num_75: double (nullable = true)
 |-- num_985: double (nullable = true)
 |-- num_100: double (nullable = true)
 |-- num_unq: double (nullable = true)
 |-- total_secs: double (nullable = true)



In [7]:
df_logs.groupBy("safra").count().orderBy("safra").show(30, truncate=False)

+------+-------+
|safra |count  |
+------+-------+
|201501|937789 |
|201502|933040 |
|201503|944739 |
|201504|939930 |
|201505|924216 |
|201506|916862 |
|201507|871491 |
|201508|920129 |
|201509|903194 |
|201510|1012953|
|201511|1041975|
|201512|1039271|
|201601|1076712|
|201602|1041248|
|201603|1048941|
|201604|1042406|
|201605|1056491|
|201606|1081181|
|201607|1102807|
|201608|1103078|
|201609|1112601|
|201610|1139089|
|201611|1183088|
|201612|1135573|
|201701|1136003|
|201702|1114164|
+------+-------+



In [8]:
df_logs.show(10, truncate=False)

+--------------------------------------------+------+------+------+------+-------+-------+-------+------------------+
|msno                                        |safra |num_25|num_50|num_75|num_985|num_100|num_unq|total_secs        |
+--------------------------------------------+------+------+------+------+-------+-------+-------+------------------+
|SwlrSivYHoKF9V5wm1YYYAnjHpd9y3OPjI9rDUhGJ3k=|201701|121.0 |28.0  |14.0  |29.0   |704.0  |827.0  |184606.903        |
|rE5wSmHEF1Dhu55zhkiGB1HvotdlSHcIMGXv6VcqO2A=|201605|26.0  |2.0   |5.0   |6.0    |462.0  |256.0  |119439.485        |
|hx+cyaQ/Jcdr/Z5foa/Cn0PXUzC/F7QO/NQvWQS1Qtc=|201611|161.0 |71.0  |49.0  |34.0   |668.0  |891.0  |204791.242        |
|53QW6B70J23X2UCvxaaUppjyE0b6X9nzP79W4huZv+Q=|201502|37.0  |9.0   |3.0   |9.0    |408.0  |447.0  |101186.041        |
|/0S1N/oRyxGLZlzxnW5rOjfo0ZAls9EH23ahuDNuqz8=|201506|205.0 |49.0  |23.0  |21.0   |225.0  |489.0  |69957.524         |
|qB/zteXKaOk3hzFCoIUD6wrTp57hnreDX4Vvon25MfM=|201509|52.

First assumptions:

* The variables "num_percent" are related to songs that can be REPEATED, which means, they only interfer in "num_unq" variable with the rule that their sum cannot be below num_unq --> it would not make sense since if we listen a new song we add another value to num_unq and add this one value in one of num_percent variables AND if we listen again to this song we DO NOT add in num_unq, BUT add in one of num_percent.

In [20]:
df_members.groupBy("city").count().orderBy(F.desc("count")).show(21, truncate=False)

+----+--------+
|city|count   |
+----+--------+
|1   |41955263|
|5   |4271348 |
|13  |3650065 |
|4   |2730740 |
|22  |2328925 |
|15  |2123349 |
|6   |1492818 |
|14  |1004416 |
|12  |740584  |
|9   |538369  |
|11  |529654  |
|8   |506488  |
|18  |424839  |
|10  |362007  |
|21  |342973  |
|17  |308525  |
|3   |303161  |
|7   |135492  |
|16  |57166   |
|20  |47238   |
|19  |13826   |
+----+--------+



## transactions

### dictionary

Definition: transactions of users up until 3/31/2017.

* msno: user id
* payment_method_id: payment method
* payment_plan_days: length of membership plan in days
* plan_list_price: in New Taiwan Dollar (NTD)
* actual_amount_paid: in New Taiwan Dollar 
(NTD)
* is_auto_renew: automatic renovation on/off
* transaction_date: format %Y%m%d
* membership_expire_date: format %Y%m%d
* is_cancel: whether or not the user canceled the 
membership in this transaction.



### general info

In [4]:
df_transactions.count()

20712225

In [7]:
df_transactions.printSchema()

root
 |-- msno: string (nullable = true)
 |-- payment_method_id: string (nullable = true)
 |-- payment_plan_days: string (nullable = true)
 |-- plan_list_price: string (nullable = true)
 |-- actual_amount_paid: string (nullable = true)
 |-- is_auto_renew: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- membership_expire_date: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- safra: integer (nullable = true)



In [9]:
df_transactions.show(10, truncate=False)

+--------------------------------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+------+
|msno                                        |payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|safra |
+--------------------------------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+------+
|+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=|38               |410              |1788           |1788              |0            |20151121        |20170104              |0        |201511|
|+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=|41               |30               |149            |149               |1            |20150526        |20150626              |0        |201505|
|+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=|41       

### plan_list_price + actual_amount_paid

In [16]:
df_transactions.groupBy("plan_list_price", "actual_amount_paid").count().orderBy(F.desc("count")).show(40)

+---------------+------------------+--------+
|plan_list_price|actual_amount_paid|   count|
+---------------+------------------+--------+
|            149|               149|11233604|
|             99|                99| 4774736|
|            129|               129| 1096720|
|              0|               149|  764403|
|            180|               180|  656705|
|              0|                 0|  562167|
|            149|                 0|  484549|
|            150|               150|  360929|
|            149|               119|  270324|
|            894|               894|  107881|
|           1788|              1788|   78940|
|            100|               100|   75569|
|            536|               536|   42850|
|            119|               119|   29195|
|              0|               129|   25410|
|              0|               119|   23335|
|            480|               480|   22736|
|              0|               150|   12462|
|           1599|              159

Nem tudo o que deveriam pagar foi efetivamente pago em alguns casos, bem como aparentemente houveram pagamentos que nao deveriam ter sido executados.

### payment_method_id

In [10]:
df_transactions.groupBy("payment_method_id").count().orderBy(F.desc("count")).show(40)

+-----------------+--------+
|payment_method_id|   count|
+-----------------+--------+
|               41|11026911|
|               40| 2160126|
|               38| 1618337|
|               39| 1437059|
|               37|  991185|
|               36|  816661|
|               34|  728384|
|               35|  492261|
|               33|  402388|
|               31|  248960|
|               30|  150972|
|               32|  144037|
|               29|  108713|
|               28|   92752|
|               27|   60532|
|               23|   40594|
|               19|   32045|
|               20|   27977|
|               21|   22839|
|               22|   19973|
|               18|   15709|
|               14|   13591|
|               24|   12620|
|               25|   11741|
|               16|   10301|
|               13|    6305|
|               17|    5081|
|               12|    3796|
|               26|    2795|
|               11|    2114|
|               15|    1464|
|             

In [15]:
df_transactions.filter(F.col("payment_plan_days").isin(30, 31)).groupBy("payment_method_id", "plan_list_price", "payment_plan_days").count().orderBy(F.desc("count")).show(100)

+-----------------+---------------+-----------------+-------+
|payment_method_id|plan_list_price|payment_plan_days|  count|
+-----------------+---------------+-----------------+-------+
|               41|            149|               30|4909578|
|               41|             99|               30|4774736|
|               40|            149|               30|1749344|
|               38|            149|               30|1410136|
|               39|            149|               30|1132316|
|               41|            129|               30|1052734|
|               37|            149|               30| 819419|
|               36|            180|               30| 548264|
|               34|            149|               30| 514645|
|               40|            149|               31| 347336|
|               33|            149|               30| 279289|
|               36|            150|               30| 260519|
|               39|            149|               31| 229024|
|       

### is_cancel

In [4]:
df_transactions.groupBy("is_cancel").count().show()

+---------+--------+
|is_cancel|   count|
+---------+--------+
|        0|20031271|
|        1|  680954|
+---------+--------+



### payment_plan_days

In [ ]:
df_transactions.groupBy("payment_plan_days").count().orderBy(F.desc("count")).show()

+-----------------+--------+
|payment_plan_days|   count|
+-----------------+--------+
|               30|18251936|
|                0|  848204|
|               31|  743859|
|                7|  517635|
|              195|  107816|
|              410|   78425|
|              180|   51019|
|              100|   23828|
|               10|   22538|
|               90|   11518|
|              395|   10351|
|              120|    9882|
|               60|    6460|
|               14|    6111|
|              200|    5742|
|              360|    5275|
|                1|    2875|
|              400|    1791|
|              450|    1238|
|               45|    1081|
+-----------------+--------+
only showing top 20 rows



In [32]:
print("Publico com pacote de assinatura mensal:", str(df_transactions.filter(F.col("payment_plan_days").isin(30, 31)).count()))
print("Representatividade (%) do publico total:", (df_transactions.filter(F.col("payment_plan_days").isin(30, 31)).count() / df_transactions.count()) * 100)

Publico com pacote de assinatura mensal: 18995795
Representatividade (%) do publico total: 91.71296178947458


In [30]:
print("Publico com pacote de assinatura inferior ao mensal:", str(df_transactions.filter(F.col("payment_plan_days") < 30).count()))
print("Representatividade (%) do publico total:", (df_transactions.filter(F.col("payment_plan_days") < 30).count() / df_transactions.count()) * 100)

Publico com pacote de assinatura inferior ao mensal: 1398223
Representatividade (%) do publico total: 6.750713648581937


In [ ]:
print("Publico com pacote de assinatura superior ao mensal:", str(df_transactions.filter(F.col("payment_plan_days") > 31).count()))
print("Representatividade (%) do publico total:", str((df_transactions.filter(F.col("payment_plan_days") > 31).count() / df_transactions.count()) * 100))

Publico com pacote de assinatura superior ao mensal: 318207
Representatividade (%) do publico total: 1.5363245619434898


In [20]:
91.71296178947458 + 6.750713648581937 + 1.5363245619434898

100.0

Ideia, verificar se faz sentido -> Construir tres publicos para o modelo: menor de 1 mes, mensal e longa data

Aspecto pra verificar: o cara mantem permanentemente o tipo de pagamento em dias, ou altera ao longo da permanencia dele na base?

In [10]:
df_transactions.filter(F.col("msno").isin("+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=")).select("safra", "payment_plan_days").orderBy("safra").show(10)

+------+-----------------+
| safra|payment_plan_days|
+------+-----------------+
|201501|               30|
|201502|               30|
|201503|               30|
|201504|                0|
|201505|               30|
|201506|               30|
|201507|               30|
|201508|               30|
|201509|               30|
|201510|               30|
+------+-----------------+
only showing top 10 rows



### actual_amount_paid

In [5]:
df_transactions = df_transactions.withColumn("actual_amount_paid", F.col("actual_amount_paid").cast("float"))

In [6]:
df_transactions.select("actual_amount_paid").summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

+-------+------------------+
|summary|actual_amount_paid|
+-------+------------------+
|  count|          20712225|
|   mean|142.83468555406287|
| stddev|133.60944276809502|
|    min|               0.0|
|    25%|              99.0|
|    50%|             149.0|
|    75%|             149.0|
|    max|            2000.0|
+-------+------------------+



In [7]:
df_transactions.groupBy("actual_amount_paid").count().orderBy(F.desc("actual_amount_paid")).show()

+------------------+------+
|actual_amount_paid| count|
+------------------+------+
|            2000.0|   119|
|            1825.0|     2|
|            1802.0|     1|
|            1800.0|     1|
|            1799.0|     1|
|            1788.0| 82809|
|            1599.0| 11991|
|            1520.0|    13|
|            1200.0|  5313|
|            1150.0|    95|
|            1000.0|   741|
|             930.0|  5063|
|             894.0|111410|
|             890.0|     1|
|             849.0|     1|
|             800.0|    11|
|             799.0|  6422|
|             760.0|     2|
|             699.0|   887|
|             600.0|     3|
+------------------+------+
only showing top 20 rows



In [13]:
((df_transactions.filter(F.col("actual_amount_paid") > 149).count()) / (df_transactions.count())) * 100

6.521515674921454

Por representar aproximadamente 6,5% do total de transacoes, talvez faca sentido considerar como ponto de alavancagem e remover?

PENDENTE: aplicar a formula para capturar casos de alavancagem e ver se a quantidade diminui. O filtro atual considera apenas o valor mais comum ate 75% dos dados ordenados.

## members

### dictionary

Definition: User information. Note that not every user in the dataset is available.

* msno
* city
* bd: age. Note: this column has outlier values ranging 
from -7000 to 2015, please use your judgement.
* gender
* registered_via: registration method
* registration_init_time: format %Y%m%d




### general info

In [ ]:
df_members.count()

63867246

In [11]:
df_members.printSchema()

root
 |-- msno: string (nullable = true)
 |-- safra: string (nullable = true)
 |-- registration_init_time: string (nullable = true)
 |-- city: string (nullable = true)
 |-- bd: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: string (nullable = true)
 |-- is_ativo: integer (nullable = true)



In [10]:
df_members.show(10, truncate=False)

+--------------------------------------------+------+----------------------+----+---+------+--------------+--------+
|msno                                        |safra |registration_init_time|city|bd |gender|registered_via|is_ativo|
+--------------------------------------------+------+----------------------+----+---+------+--------------+--------+
|+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=|201612|20140927              |1   |0  |NULL  |7             |1       |
|++/AwGzubug3gT6J+0STBGMdWKxaM+UFZTI8Tcmq4To=|201607|20150322              |1   |0  |NULL  |9             |0       |
|++/Gw1B9K+XOlB3hLTloeUK2QlCa2m+BJ8TrzGf7djI=|201601|20121217              |15  |32 |male  |3             |1       |
|++02XbtviomSxcIBUHMOiJkjRxdicTXSfiVqLdsr5lo=|201603|20131112              |14  |21 |male  |7             |0       |
|++0O0Bq04sB/9ZcOS+pajpYL2Hin9jCqnc/8bKzKFuE=|201610|20141021              |5   |33 |male  |3             |0       |
|++2AQgVgYUAqJDw684tbDqDffUeKhqydyQmbr8lz9lQ=|201608|20150416   

In [18]:
df_members.select("registered_via").distinct().show(50, truncate=False)

+--------------+
|registered_via|
+--------------+
|7             |
|11            |
|3             |
|8             |
|16            |
|5             |
|17            |
|6             |
|19            |
|9             |
|1             |
|10            |
|4             |
|13            |
|14            |
|2             |
|-1            |
+--------------+



In [21]:
spark.stop()